In [6]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import IBMQ
from qiskit import pulse            
from qiskit.pulse import pulse_lib 
from qiskit import assemble
from qiskit.tools.monitor import job_monitor

from init import (
    backend as init_backend
    ,backend_open_pulse as init_open_pulse
    ,provider
)

from utils import (
    make_drive_pulse, make_meas_pulse
    ,freq_sweep_schedule
    ,sweep_program
    ,qbit_freqs
)


In [ ]:
anl = provider('anl')

In [10]:
for back in anl.backends():
    conf = back.configuration()
    print(back, conf.open_pulse)
    


ibmq_qasm_simulator False
ibmqx2 False
ibmq_16_melbourne False
ibmq_poughkeepsie True
ibmq_boeblingen True
ibmq_vigo False
ibmq_ourense False
ibmq_johannesburg True
ibmq_rochester False


In [67]:
#backend = init_open_pulse()
backend = init_backend('ibmq_johannesburg', provider_name='anl')

Initializing IBMQ ibmq_johannesburg


In [68]:
frequency_sweep_program = sweep_program(backend, frequencies=None)

Estimated frequency for qbit 0 is 4.664856171304275
Sampling time: 3.5555555555555554 ns


In [69]:
job = backend.run(frequency_sweep_program)

In [90]:
#job_monitor(job)
job.status()
#job.error_message()

<JobStatus.QUEUED: 'job is queued'>

In [97]:
from qiskit.providers.ibmq.job.ibmqjob import IBMQJob

Client = (job._api.__class__)
proj_url = anl.credentials.url
ws_url = anl.credentials.websockets_url
token = anl.credentials.token
job_id = None
# api instance
client = Client(token, proj_url, ws_url)

j_ = IBMQJob(anl, job_id, client, qobj=frequency_sweep_program)

In [95]:
j_.status()

<JobStatus.INITIALIZING: 'job is being initialized'>

In [ ]:
def unpack_results(results):
    sweep_values = []
    for i in range(len(results.results)):
        # Get the results from the ith experiment
        res = results.get_memory(i)
        # Get the results for `qubit` from this experiment
        sweep_values.append(res[qbit])
    return sweep_values

results = job.result(timeout=120)

Job Status: job is queued (30)      

In [7]:
frequency_sweep_results = job.result(timeout=120) # timeout parameter set to 120 seconds
print(frequency_sweep_results)


plt.scatter(frequencies, sweep_values, color='black') # plot real part of sweep values
plt.xlim([min(frequencies), max(frequencies)])
plt.xlabel("Frequency [GHz]")
plt.ylabel("Measured signal [a.u.]")
plt.show()

### Fit results

from scipy.optimize import curve_fit

def fit_function(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

fit_params, y_fit = fit_function(frequencies,
                                 sweep_values, 
                                 lambda x, A, q_freq, B, C: (A / np.pi) * (B / ((x - q_freq)**2 + B**2)) + C,
                                 [-2e10, 4.975, 1, 3e10] # initial parameters for curve_fit
                                )
print(fit_params)

plt.scatter(frequencies, sweep_values, color='black')
plt.plot(frequencies, y_fit, color='red')
plt.xlim([min(frequencies), max(frequencies)])

plt.xlabel("Frequency [GHz]")
plt.ylabel("Measured Signal [a.u.]")
plt.show()

print("Fit center:",fit_params[1])
print("IBM center:", center_est_freq)
print("IBM error:", (fit_params[1]-center_est_freq)/fit_params[1])

NameError: name 'job' is not defined